In [30]:
import warnings
warnings.filterwarnings("ignore")
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from os import listdir
from os.path import isfile, join

In [3]:
k.set_image_dim_ordering('tf')
img_width = 64
img_height = 64
train_data_dir = "data/train"
validation_data_dir = "data/valid"
nb_train_samples = 25
nb_validation_samples = 8
batch_size = 1
epochs = 50
num_of_classes = 2

In [17]:
# Loading base model.
model = VGG16(weights='imagenet', include_top=False,input_shape = (img_width, img_height,3))

In [5]:
for layer in model.layers[:-5]:
    layer.trainable = False

In [22]:
# ADDING FC LAYERS ON TOP OF VGG16..
top_layers = model.output
top_layers = Flatten(input_shape=model.output_shape[1:])(top_layers)
top_layers = Dense(128, activation="relu",input_shape=(num_of_classes,))(top_layers)
top_layers = Dropout(0.2)(top_layers)
top_layers = Dense(64, activation="relu",input_shape=(num_of_classes,))(top_layers)
top_layers = Dropout(0.2)(top_layers)
top_layers = Dense(num_of_classes, activation="relu",input_shape=(num_of_classes,))(top_layers)
top_layers = Dense(num_of_classes, activation="softmax")(top_layers)

In [23]:
model_final = Model(input = model.input, output = top_layers)

In [24]:
model_final.compile(loss = "binary_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [9]:
# Initialize test and training data
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

In [26]:
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")


validation_generator = test_datagen.flow_from_directory(
validation_data_dir, target_size = (img_height, img_width), class_mode = "categorical")

Found 27 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


In [13]:
checkpoint = ModelCheckpoint("vgg16_12.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [27]:
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early]
)

Epoch 1/10
24/25 [===========================>..] - ETA: 0s - loss: 0.5329 - acc: 0.8333
Epoch 00001: val_acc did not improve
25/25 [==============================] - 11s 440ms/step - loss: 0.5382 - acc: 0.8400 - val_loss: 0.6791 - val_acc: 0.5833
Epoch 2/10
24/25 [===========================>..] - ETA: 0s - loss: 0.5895 - acc: 0.6250
Epoch 00002: val_acc did not improve
25/25 [==============================] - 9s 367ms/step - loss: 0.5695 - acc: 0.6400 - val_loss: 0.5098 - val_acc: 0.8333
Epoch 3/10
24/25 [===========================>..] - ETA: 0s - loss: 0.6430 - acc: 0.7500
Epoch 00003: val_acc did not improve
25/25 [==============================] - 9s 367ms/step - loss: 0.6437 - acc: 0.7600 - val_loss: 0.5513 - val_acc: 0.8125
Epoch 4/10
24/25 [===========================>..] - ETA: 0s - loss: 0.5229 - acc: 0.8333
Epoch 00004: val_acc did not improve
25/25 [==============================] - 9s 366ms/step - loss: 0.5297 - acc: 0.8400 - val_loss: 0.5354 - val_acc: 0.8333
Epoch 5/10


In [28]:
# Okay, Training is done. Now, let's do testing.

In [29]:
test_path1 = "data/test/not"
test_path2 = "data/test/sure"

In [31]:
no = [f for f in listdir(test_path1) if isfile(join(test_path1, f))]
sure = [f for f in listdir(test_path2) if isfile(join(test_path2, f))]
print(no, sure)

['nt1.jpg', 'nt2.jpg', 'nt3.jpg', 'nt4.jpg'] ['st1.jpg', 'st2.jpg', 'st3.jpg', 'st4.jpg']


In [32]:
for i in no:
    test_image = image.load_img(test_path1 + "/" + i, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model_final.predict(test_image)
    if result[0][0] == 1:
        prediction = 'not'
    else:
        prediction = 'sure'
    print(prediction)

not
not
not
not


In [35]:
#Whoa, 100% Accurate on TEST DATA on "not torn" shirts.

In [34]:
for i in sure:
    test_image = image.load_img(test_path2 + "/" + i, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model_final.predict(test_image)
    if result[0][0] == 1:
        prediction = 'not'
    else:
        prediction = 'sure'
    print(prediction)

not
sure
sure
sure


In [36]:
# 75% On "torn shirts".

In [37]:
"""
SO, OVERALL, BY LESS THAN 50 IMAGES OF TRAINING, MODEL PERFORMS GOOD.
"""

'\nSO, OVERALL, BY LESS THAN 50 IMAGES OF TRAINING, MODEL PERFORMS GOOD.\n'